In [1]:
import ee
import numpy as np
import pandas as pd

In [2]:
ee.Authenticate()


True

In [3]:
ee.Initialize(project='ee-apr160')
print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


In [4]:
#df_data = pd.read_csv('data/2023.csv',encoding = "ISO-8859-1")
#df_data = df_data[['Latitud','Longitud']].drop_duplicates().reset_index(drop=True)
#df_data = df_data.iloc[1628:]
#df_data = pd.read_csv('missing.csv',encoding = "ISO-8859-1")
df_data = pd.read_csv('data/missing.csv',encoding = "ISO-8859-1")
df_data


,Latitud,Longitud
0,21.51200,-87.89104
1,21.27910,-89.26435
2,21.50130,-88.12003
3,21.50140,-88.28179
4,21.51210,-88.21687
...,...,...
65,21.55655,-88.07202
66,21.56558,-88.09142
67,21.57153,-88.16399
68,21.57448,-88.11790


In [5]:
def add_date(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

def save_coordinate(Sentinel_data,row, feature_list,name_folder):
    features = []
    poi_geometry = ee.Geometry.Point([row['Longitud'], row['Latitud']])
    poi_properties = dict(row)
    poi_feature = ee.Feature(poi_geometry, poi_properties)
    features.append(poi_feature)
    ee_fc = ee.FeatureCollection(features) 


    def rasterExtraction(image):
        feature = image.sampleRegions(
            collection = ee_fc, # feature collection here
            scale = 10 # Cell size of raster
        )
        return feature
    
    results = Sentinel_data.filterBounds(ee_fc).select(feature_list).map(add_date).map(rasterExtraction).flatten()

    # You can modify this for better optimization
    column_df = list(df_data.columns)
    column_df.extend(feature_list + ['date'])
    
    nested_list = results.reduceColumns(ee.Reducer.toList(len(column_df)), column_df).values().get(0)
    data = nested_list.getInfo()
    df_final = pd.DataFrame(data,columns=column_df)
    df_final = df_final.groupby(["date","Latitud","Longitud"]).agg(
                Psurf_f_inst=('Psurf_f_inst', 'mean'),
                Qair_f_inst=('Qair_f_inst', 'mean'),
                Rainf_tavg=('Rainf_tavg', 'mean'),
                Tair_f_inst=('Rainf_tavg', 'mean'),
                Wind_f_inst=('Wind_f_inst', 'mean'),
                RootMoist_inst=('RootMoist_inst', 'mean')) \
                .reset_index()
               

    df_final.to_csv('gee-coordinates-2023/'+name_folder+'/' + str(row['Latitud']) + '_' + str(row['Longitud']) + '.csv')
   

def get_data(source,from_date,to_date,feature,name_folder):
    Sentinel_data = ee.ImageCollection(source) \
    .filterDate(from_date,to_date) \
    .map(add_date)

    for index, row in df_data.iterrows():
        save_coordinate(Sentinel_data,row,feature,name_folder)
        print('.',end='')

In [16]:
get_data('NASA/ORNL/DAYMET_V4',"2022-09-20","2023-12-31",['prcp','srad','tmax','tmin','vp'],'DAYMET_V4')

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [ ]:
get_data('IDAHO_EPSCOR/GRIDMET',"2021-09-20","2022-12-31",['pr','sph','th','vs'],'IDAHO_EPSCOR')

In [6]:
get_data('NASA/GLDAS/V021/NOAH/G025/T3H',"2022-09-20","2023-12-31",['Psurf_f_inst','Qair_f_inst','Rainf_tavg','RootMoist_inst','Tair_f_inst','Wind_f_inst'],'SOIL')


......................................................................